# Rubik's Cube SAT Solver trial

## Modélisation: 

Le problème est un modèle dynamique, comme le cube prend différents états au cours du temps, suivant les actions qui sont choisies.
Il est donc nécessaire de prendre en compte la dimension temporelle dans notre problème, comme nous ne cherchons pas à déterminer un état final (car on connait l'état final), mais plutôt à déterminer les actions à effectuer pour passer d'un état initial à un état final.

Il faut donc modéliser d'une part les états du cube  et d'autre part les actions choisies à chaques étapes.

### Modélisation du cube : 

Pour chaque étapes, on modélise le cube en numérotant ses faces de 1 à 6, et en numérotant les cases de 1 à 4 pour chaques faces. Il nous faut aussi encoder la couleur de chaque case pour cela plusieurs choix s'offrent à nous:

![Rubik's Cube faces](./images/rubikscube1.png)
![Rubik's Cube facettes](./images/rubikscube2.png)

- En utilisant 6 variables pour chaques faces, chacune encodant la couleur. Alors seulement une variable parmis les 6 pourrat être vraie, et les autres fausses (c'est une contrainte). Cela donne $6*4*6=144$ variables pour chaques étapes.

- ou en encodant la couleur sur 3 bits, ce qui donne 8 couleurs possibles, et en interdisant 2 états pour réduire le nombre à 6 couleurs possible. Cela donne $6*4*3=72$ variables pour chaques étapes.

Pour commencer nous utiliserons la première méthode à 6 couleurs pour simplifier l'implémentation, et nous pourrons essayer la deuxième méthode si le temps de calcul est trop long.

### Modélisation des actions :

Pour modéliser les action du cubes il faut en premier lieu dénombrer le nombre d'actions possibles. Il y a en tout 18 mouvements posssibles, pour chaque face on peut faire une rotation de 90° dans le sens horaire ou anti-horaire, ou une rotation de 180°, ce qui donne $6*3=18$ actions possibles.

En réalité beaucoup sont redondantes. Par exemple, si on fait une rotation de 90° dans le sens horaire de la face 1, cela revient à faire une rotation de 270° dans le sens anti-horaire de la face 1, ou une rotation de 90° dans le sens anti-horaire de la face 6. On peut réduire le nombre d'actions possible à différents niveau :

- 3 actions : rotation de 90° dans le sens horaire, pour la face du haut, du devant et de gauche (les faces 1,2,3). avec ces 3 actions on peut réaliser toutes les actions, c'est la modélisation minimale. Cependant cela ne nous permet pas de trouver le nombre d'actions minimales pour résoudre le cube, car il faudra plusieur actions répétés pour réaliser d'autres actions.

- 6 actions : pareil en rajoutant les rotations de 90° dans le sens antihoraire pour chaque face

- 9 actions : la manière optimale, en rajoutant les rotations de 180° pour les 3 faces. Avec ces 9 actions on peut réaliser toutes les actions possibles en 1 coup.

Nous utiliserons donc 9 actions pour modéliser les actions du cube, soit les rotations de 90° dans les 2 sens et les rotations de 180° pour chaque face.

Pour encoder ces 9 actions dans des variables propositionnelles, il existe plusieur manières:

- avec 9 variables propositionnelles, une pour chaque action

- avec 6 variables propositionnelle, 3 pour le choix de la face à tourner et 3 pour le choix du mouvement à réaliser (90° horaire, 90° anti-horaire, 180°)

- avec 4 variables en encodant le choix de la face à tourner sur 2 bits et le choix du mouvement sur 2 bits. ou en encodant directement le choix du mouvement sur 4 bits. Cette méthode est la plus compacte, mais elle est bien plus difficile à implémenter.

Nous utiliserons la première ou la 2eme méthode selon ce qui semble le plus simple à implémenter dans un premier temps.

### Modélisation de la résolution du cube : 

Il est important de remarquer que 

2 options s'offrent encore à nous:

- Proposition 1: on fixe le nombre d'étape, en commenceant par 1 étape. On implémente les contraintes CNF pour que le problème soit satisfait à la dernière étape. On résout le problème grâce au solveur, et si le problème est insatisfiable on recommence en incrémentant le nombre d'étape de 1, jusqu'à résolution du problème. Cette méthode nous donnera alors la solutions la plus courte possible.

- Proposition 2 : on fixe le nombre d'étape à 11, comme on sait que le cube est résolvable en 11 étapes maximum (littérature). Et on ajoute 11 variables au problème une variable i était vrai si à l'étape i le cube est résolu. Il faudra alors au moins 1 des variables i vraie pour que le problème soit satisfait. Cette modélisation ne donnera pas forcément la solution la plus courte, sauf peut être en ajoutant des contraintes supplémentaires (on ne voit pas encore lesquelles).

Pour avoir la solution optimale, nous allons nous concentrer sur le choix 1 pour l'instant.

Pour la contrainte de résolution, on force les 6 faces à avoir une unique couleur pour chacune des 6 cases. On peut aussi ajouter des contraintes pour forcer les couleurs des cases à être celles de l'état final, mais ça surcontraindrait le problème car la configuration finale est possible pour 24 état différents selons comment on tourne le cube, hors on cherche juste à résoudre le cube et pas un état final précis.


## Implémentation:


### Les variables propositionnelles:

In [68]:
# implémentation de la modélisation du probmème

import numpy as np 

t_max = 11 # (pour l'instant)

def variables(n_steps):
    n_faces = 6
    n_cases = 4
    n_couleurs = 6
    R = np.arange(1, (n_steps+1)*n_faces*n_cases*n_couleurs + 1).reshape((n_steps+1), n_faces, n_cases, n_couleurs)

    # les actions (3 mouvements et 3 faces)
    n_mouvements = 3
    n_faces = 3
    M = np.arange(1, t_max*n_mouvements  + 1).reshape(t_max, n_mouvements)
    M += np.max(R)
    F = np.arange(1, t_max*n_faces  + 1).reshape(t_max, n_faces)
    F += np.max(M)
    
    return R, M, F
    
R, M, F = variables(t_max)

print("on s'assure que 2 variables n'ont pas le même nombre: ")
print("R :",R.min(),"->", R.max())
print("A :",M.min(),"->", M.max())
print("F :",F.min(),"->", F.max())
print("on a donc", F.max(), "variables propositionnelles pour notre modélisation.")



on s'assure que 2 variables n'ont pas le même nombre: 
R : 1 -> 1728
A : 1729 -> 1761
F : 1762 -> 1794
on a donc 1794 variables propositionnelles pour notre modélisation.


### Contraintes d'unicité des mouvements:

In [69]:
# CONTRAINTES DE BASE POUR LE CUBE

# en général on utilisera 
# t pour le numéro de l'étape
# f pour la face
# i pour le numéro de la case
# c pour la couleur

def contrainte_unique_couleurs(R):
    
    t_max, n_faces, n_cases, n_couleurs = R.shape

    basic_R_constraints = []

    # chaque sommet doit être colorié d'au moins une couleur
    for t,f,i in np.ndindex((t_max, n_faces, n_cases)):
        basic_R_constraints.append([int(R[t,f,i,c]) for c in range(6)])

    # chaque sommet doit être colorié d'au plus une couleur
    for t,f,i in np.ndindex((t_max,n_faces,n_cases)):
        for k in range(n_couleurs):
            for l in range(k+1,n_couleurs):
                basic_R_constraints.append([-int(R[t,f,i,k]), -int(R[t,f,i,l])])
    
    print("nombre de clauses pour l'unicité des couleurs:", len(basic_R_constraints))
    return basic_R_constraints
            
basic_R_clauses_color = contrainte_unique_couleurs(R)


nombre de clauses pour l'unicité des couleurs: 4608


### Contraintes d'unicité des mouvements:

In [70]:
# CONTRAINTES DE BASE POUR LES MOUVEMENTS

def contraintes_unique_mouvement(M,F):
    t_max, n_mouvements = M.shape
    t_max, n_faces = F.shape

    basic_moves_constraints = []

    # il doit y avoir au moins un mouvement sur une face à chaque étape,
    for t in range(t_max):
        basic_moves_constraints.append([int(M[t,k]) for k in range(n_mouvements)])
        basic_moves_constraints.append([int(F[t,k]) for k in range(n_faces)])
        
        
    # il doit y avoir au plus un mouvement sur une face à chaque étape
    for t in range(t_max):
        for k in range(3):
            for l in range(k+1,3):
                basic_moves_constraints.append([-int(M[t,k]), -int(M[t,l])])
                basic_moves_constraints.append([-int(F[t,k]), -int(F[t,l])])
    
    print("nombre de contraintes pour l'unicité des mouvements:", len(basic_moves_constraints))
    return basic_moves_constraints
            
basic_R_clauses_move = contraintes_unique_mouvement(M,F)

nombre de contraintes pour l'unicité des mouvements: 88


### Contraintes de résolution du cube:

In [71]:
# Contraite de résolution du problème

# à la dernière étape, pour chaque face, 
# les 3 premières facettes doivent être de la même couleur que la 4ème

# on peut mettre la contrainte sous forme d'égalité, mais cela nous rajoutera 2 contrainte par égalité,
# à la place ou peut simplement la mettre sous forme d'implication: 
# "pour 2 facettes A et B, pour toutes couleurs k, si A[k] alors B[k]"
# comme chaque case est déjà contrainte à n'avoir qu'une seule et même couleur, cela revient au même.
# on utilisera donc toujours l'implication au lieu de l'égalité, ce qui divise le nombre de contraintes par 2.

# on peut le prouver assez facilement:
# pour k tel que Ak = 1,  Ak => Bk , alors Bk = 1 , et ensuite pour tout f !=k , Af=0 & Bf=0,

def equal_to(a,b):
    return [[-a,b],[a,-b]]

def implies(a,b):
    return [[-a,b]]

def contraintes_de_resolution(R):
    t_max, n_faces, n_cases, n_couleurs = R.shape
    resolution_clauses = []
    for f in range(n_faces):
        for c in range(n_cases-1):
            for k in range(n_couleurs):
                resolution_clauses += implies(R[t_max-1,f,c,k], R[t_max-1,f,n_cases-1,k])
    print("nombre de contraintes de résolution:", len(resolution_clauses))
    return resolution_clauses

resolution_constraints = contraintes_de_resolution(R)

nombre de contraintes de résolution: 108


### Contraintes de mouvement

Pour appliquer les contraintes de mouvements (cf de rotation), il va faloir définir chacun des mouvements pour chacunes des faces, pour nous simplifier le travail, on remodélise les 3 faces de la même façon pour n'avoir à définir les 3 mouvements différents que pour une seule face.

on renomme les facettes de la façon suivante pour chaque face (par exemple pour la face 2):

![Rubik's Cube facettes](./images/faces_renum.png)

On rappelle la modélisation du cube : 

![Rubik's Cube faces](./images/rubikscube1.png)
![Rubik's Cube facettes](./images/rubikscube2.png)

In [ ]:
# rotation horaire face 1

# on cherche à traduire des implications découlant des actions choisies

# exemple sens horaire : 
# F(t,1) \and M(t,1) => R(t,1,1,:) = R(t+1,1,2,:)
# on peut déjà le transformer en une implication comme expliqué plus haut:
# F(t,1) \and M(t,1) => ( R(t,1,1,:) => R(t+1,1,2,:) ) 
# on peut la transformer en forme normale conjonctive:
# ( -F(t,1) \or -M(t,1) \or -R(t,1,1,:) \or R(t+1,1,2,:) )

# les mouvements sont numérotés comme suis:
move = {
    "rotate_90" : 0,
    "rotate_90_inv" : 1,
    "rotate_180" : 2,
}

adjacences = {
    0 : ( (3,0), (3,1), (2,0), (2,1), (1,0), (1,1), (4,0), (4,1) ),
    1 : ( (0,3), (0,2), (2,1), (2,3), (5,0), (5,1), (4,2), (4,0) ),
    2 : ( (0,2), (0,0), (3,1), (3,3), (5,2), (5,0), (1,2), (1,0) ),
}


def rotate_90(f, t, R, M, F):
    C = []
    # face principale
    implications = [(0,1), (0,2), (2,3), (3,1)]
    for c1, c2 in implications:
        for k in range(6):
            C.append([-F[t,f], -M[t,move["rotate_90"]], -R[t,f,c1,k], R[t+1,f,c2,k]])
    # facettes adjacentes:
    ordre = adjacences[f]
    for i in range(len(ordre)):
        f1, c1 = ordre[i]
        f2, c2 = ordre[ (i-2) % 8]
        for k in range(6):
            C.append([-F[t,f], -M[t,move["rotate_90"]], -R[t,f1,c1,k], R[t+1,f2,c2,k]]) 
    return C
    
        
def rotate_90_inv(f, t, R, M, F):
    C = []
    # face principale
    implications = [(1,0), (1,3), (3,2), (2,0)]
    for c1, c2 in implications:
        for k in range(6):
            C.append([-F[t,f], -M[t,move["rotate_90_inv"]], -R[t,f,c1,k], R[t+1,f,c2,k]])
    # facettes adjacentes:
    ordre = adjacences[f]
    for i in range(len(ordre)):
        f1, c1 = ordre[i]
        f2, c2 = ordre[ (i+2) % 8]
        for k in range(6):
            C.append([-F[t,f], -M[t,move["rotate_90_inv"]], -R[t,f1,c1,k], R[t+1,f2,c2,k]]) 
    return C
                        
def rotate_180(f, t, R, M, F):
    C = []
    # face principale
    implications = [(0,3),(3,0), (1,2), (2,1)]
    for c1, c2 in implications:
        for k in range(6):
            C.append([-F[t,f], -M[t,move["rotate_180"]], -R[t,f,c1,k], R[t+1,f,c2,k]])
    # facettes adjacentes:
    ordre = adjacences[f]
    for i in range(len(ordre)):
        f1, c1 = ordre[i]
        f2, c2 = ordre[ (i+4) % 8]
        for k in range(6):
            C.append([-F[t,f], -M[t,move["rotate_180"]], -R[t,f1,c1,k], R[t+1,f2,c2,k]]) 
    return C



def contraintes_de_mouvements(R,M,F):
    t_max, n_faces, n_cases, n_couleurs = R.shape
    n_steps, n_faces_action = F.shape
    
    mouvement_clauses = []

    for t in range(t_max-1):
        for f in range(n_faces_action):
            mouvement_clauses += rotate_90(f,t,R,M,F)
            mouvement_clauses += rotate_90_inv(f,t,R,M,F)
            mouvement_clauses += rotate_180(f,t,R,M,F)
            # les autres facettes (qui ne bougent pas):
            for face in range(n_faces):
                if face !=f:
                    for c in range(n_cases):
                        if (face,c) not in adjacences[f]:
                            for k in range(n_couleurs):
                                mouvement_clauses.append([-F[t,f], -R[t,face,c,k], R[t+1,face,c,k]])
                                
    print("nombre de contraintes:", len(mouvement_clauses))
    return mouvement_clauses

mouvement_clauses = contraintes_de_mouvements(R,M,F)


nombre de contraintes: 9504


### Contraintes de départ

Pour finaliser le jeu de contraintes, il suffit de forcer les couleurs à être dans un état initial donné. Nous devons alors créer un rubik's cube résolvable (ou non) pour tester notre solveur. Pour cela nous allons utiliser un cube résolu, et nous allons le mélanger aléatoirement pour obtenir un état initial.

Pour parler des contraintes plus précisement, il faudra une contrainte par cette, obligeant la couleur de la case à être celle de l'état initial. 

In [173]:
import random

face_2_face = {
    0: 1,  # Front
    1: 2,  # Left
    2: 0,  # Up
    3: 5,  # Right
    4: 3,  # Down
    5: 4   # Back
}

COLORS = {
    2: "🟥",  # Rouge (Up)
    0: "🟩",  # Vert (Front)
    1: "🟦",  # Bleu (Left)
    5: "🟨",  # Jaune (Down)
    3: "🟧",  # Orange (Back)
    4: "⬜"   # Blanc (Right)
}



# Création du cube 2x2x2 correctement formaté (6 faces, 2x2)
def create_cube():
    return np.array([[[i, i], [i, i]] for i in range(6)])

# Affichage amélioré du cube en format "déplié"
def print_cube(cube):
    def get_row(face, row):
        return f"{COLORS[face[row, 0]]} {COLORS[face[row, 1]]}"

    print(f"    {get_row(cube[2], 0)}")  # Haut (Up)
    print(f"    {get_row(cube[2], 1)}\n")  

    print(f"{get_row(cube[1], 0)} {get_row(cube[0], 0)} {get_row(cube[3], 0)} {get_row(cube[5], 0)}")  # Left, Front, Right, Back
    print(f"{get_row(cube[1], 1)} {get_row(cube[0], 1)} {get_row(cube[3], 1)} {get_row(cube[5], 1)}\n")  

    print(f"    {get_row(cube[4], 0)}")  # Bas (Down)
    print(f"    {get_row(cube[4], 1)}\n")  

# Rotation de 90° d’une face avec mise à jour des bords (pas besoin des autres mouvements vu qu'on ne veut pas forcément optimiser)
def rotate_face(cube, face):

    cube[face] = np.rot90(cube[face], -1)
    # Définition des indices des bords affectés
    edges = {
        0: [(2, (1, slice(None))), (1, (slice(None), 1)), (4, (0, slice(None))), (3, (slice(None), 0))],  # Front
        1: [(2, (slice(None), 0)), (5, (slice(None), 0)), (4, (slice(None), 0)), (0, (slice(None), 0))],  # Left
        2: [(5, (0, slice(None))), (3, (0, slice(None))), (0, (0, slice(None))), (1, (0, slice(None)))],  # Up
    }
    
    
    # Récupération des valeurs avant rotation
    temp = cube[edges[face][0][0]][edges[face][0][1]].copy()

    # Application de la rotation des bords
    for i in range(3):
        src_face, src_slice = edges[face][i + 1]
        dst_face, dst_slice = edges[face][i]
        cube[dst_face][dst_slice] = (
            np.flip(cube[src_face][src_slice]) if face in [0, 1, 3, 5] else cube[src_face][src_slice]
        )

    # Mise à jour de la dernière face
    last_face, last_slice = edges[face][-1]
    cube[last_face][last_slice] = np.flip(temp) if face in [0, 1, 3, 5] else temp

    return cube

# Exécution
cube = create_cube()
print("Cube initial :")
print_cube(cube)


n = 500 # Nombre de rotations aléatoires
for i in range(n):
    cube = rotate_face(cube, random.randint(0, 2))  # Rotation de la face FRONT
print("Après rotation de la face frontale :")
print_cube(cube)

# Ajouter les contraintes à notre problème

def contraintes_initiales(R, cube):
    t_max, n_faces, n_cases, n_couleurs = R.shape
    
    contraintes = []
    
    for f in range(n_faces):
        for i in range(n_cases):
            for c in range(n_couleurs):
                if cube[face_2_face[f]][i // 2][i % 2] == c:
                    contraintes.append([R[0,f,i,c]])
    return contraintes

initial_constraints = contraintes_initiales(R, cube)

print( "nombre de contraintes initiales:", len(initial_constraints) )

Cube initial :
    🟥 🟥
    🟥 🟥

🟦 🟦 🟩 🟩 🟧 🟧 🟨 🟨
🟦 🟦 🟩 🟩 🟧 🟧 🟨 🟨

    ⬜ ⬜
    ⬜ ⬜

Après rotation de la face frontale :
    🟧 🟥
    ⬜ 🟥

🟩 🟧 🟥 🟦 🟩 🟩 🟧 🟨
🟦 ⬜ 🟥 🟨 🟨 🟧 ⬜ 🟨

    🟩 🟦
    🟦 ⬜

nombre de contraintes initiales: 24


### Résolution

In [174]:
for n_steps in range (1,12):
    
    R, M, F = variables(n_steps)
    clauses = []
    clauses += contrainte_unique_couleurs(R)
    clauses += contraintes_unique_mouvement(M,F)
    clauses += contraintes_de_resolution(R)
    clauses += contraintes_de_mouvements(R,M,F)
    clauses += contraintes_initiales(R, cube)[:-12]
    
    print("nombre de variables:", F.max())
    print("nombre de clauses:", len(clauses))
    
    with open("cube_"+str(n_steps)+".cnf", "w") as f:
        f.write("c Generated by rubik_cube_cnf.py\n")
        f.write(f"c for the resolution of the cune in {n_steps} steps\n")
        f.write("c\n")
        f.write(f"p cnf {F.max()} {len(clauses)}\n")
        for clause in clauses:
            for variable in clause:
                f.write(str(variable) + " ")
            f.write("0\n")
    
    print("\n")
    
    # TODO
    # ... lancer la résolution avec le solveur et voir si c'est resolvable

nombre de clauses pour l'unicité des couleurs: 768
nombre de contraintes pour l'unicité des mouvements: 88
nombre de contraintes de résolution: 108
nombre de contraintes: 864
nombre de variables: 354
nombre de clauses: 1840


nombre de clauses pour l'unicité des couleurs: 1152
nombre de contraintes pour l'unicité des mouvements: 88
nombre de contraintes de résolution: 108
nombre de contraintes: 1728
nombre de variables: 498
nombre de clauses: 3088


nombre de clauses pour l'unicité des couleurs: 1536
nombre de contraintes pour l'unicité des mouvements: 88
nombre de contraintes de résolution: 108
nombre de contraintes: 2592
nombre de variables: 642
nombre de clauses: 4336


nombre de clauses pour l'unicité des couleurs: 1920
nombre de contraintes pour l'unicité des mouvements: 88
nombre de contraintes de résolution: 108
nombre de contraintes: 3456
nombre de variables: 786
nombre de clauses: 5584


nombre de clauses pour l'unicité des couleurs: 2304
nombre de contraintes pour l'unicité d